In [1]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
import catboost
print(catboost.__version__)
from jupyterthemes import jtplot
jtplot.style('default')
!jupyter nbextension enable --py widgetsnbextension

1.0.0
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
from catboost.datasets import amazon

(train_df, test_df) = amazon()

In [3]:
train_df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
y = train_df.ACTION
x = train_df.drop('ACTION', axis=1)

In [5]:
cat_features = list(range(0, len(x.columns)))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [6]:
print('Labels: {}'.format(set(y)))
print('Zero count = {}, One count = {}'.format(len(y)-sum(y), sum(y)))

Labels: {0, 1}
Zero count = 1897, One count = 30872


This is preety unbalanced

Now lets train our model

In [7]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100)  # we are training for 100 trees
model.fit(x, y, cat_features=cat_features, verbose=10)

# cat_features=cat_features is passing the index of the columns
# you can also give columns name if yu want to use strings
# But that will be quit a work :)

Learning rate set to 0.377604
0:	learn: 0.4528598	total: 91ms	remaining: 9s
10:	learn: 0.1744186	total: 191ms	remaining: 1.54s
20:	learn: 0.1676119	total: 295ms	remaining: 1.11s
30:	learn: 0.1652446	total: 382ms	remaining: 851ms
40:	learn: 0.1633644	total: 474ms	remaining: 682ms
50:	learn: 0.1621892	total: 569ms	remaining: 547ms
60:	learn: 0.1609164	total: 674ms	remaining: 431ms
70:	learn: 0.1594572	total: 773ms	remaining: 316ms
80:	learn: 0.1585876	total: 858ms	remaining: 201ms
90:	learn: 0.1573593	total: 967ms	remaining: 95.6ms
99:	learn: 0.1566977	total: 1.05s	remaining: 0us


In [8]:
model.predict_proba(x)

array([[0.0098, 0.9902],
       [0.0101, 0.9899],
       [0.0579, 0.9421],
       ...,
       [0.0118, 0.9882],
       [0.1891, 0.8109],
       [0.0235, 0.9765]])

This is good, if we remove the n_iteration then by default it will be set as 1000\
But, in the imbalnce  dataset sometimes you add the extra columns weight\
large weight for the rare class and small weight for the very frequent class\
or somethime you might have something else like group id like we have for some modes

If you have lot of information then yu can create a pool class

In [9]:
from catboost import Pool
pool = Pool(data=x, label=y, cat_features=cat_features)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0)

train_pool = Pool(
                    data = x_train,
                    label = y_train,
                    cat_features = cat_features
)

val_pool = Pool(
                    data = x_val,
                    label = y_val,
                    cat_features = cat_features
)

Now, there are 2 objective function available\
1. Logloss: for binary target\
2. CrossEntropy: for probabilities in target\
\
It will be selected automatically

In [11]:
model = CatBoostClassifier(
                            iterations = 5,
                            learning_rate = 0.1
)

model.fit(train_pool, eval_set = val_pool, verbose = 1)

print('Model is fittef: {}'.format(model.is_fitted()))
print('Model params: \n{}'.format(model.get_params()))

0:	learn: 0.6189489	test: 0.6187790	best: 0.6187790 (0)	total: 15.5ms	remaining: 61.9ms
1:	learn: 0.5585414	test: 0.5579742	best: 0.5579742 (1)	total: 29.4ms	remaining: 44.1ms
2:	learn: 0.5083288	test: 0.5075307	best: 0.5075307 (2)	total: 33.5ms	remaining: 22.3ms
3:	learn: 0.4661280	test: 0.4649882	best: 0.4649882 (3)	total: 37.9ms	remaining: 9.49ms
4:	learn: 0.4305277	test: 0.4291119	best: 0.4291119 (4)	total: 41.9ms	remaining: 0us

bestTest = 0.4291119343
bestIteration = 4

Model is fittef: True
Model params: 
{'iterations': 5, 'learning_rate': 0.1}


Oofcourse we wil not use iteration = 5 in real world :)

In [12]:
model = CatBoostClassifier(
                            iterations = 1000
)

model.fit(train_pool, eval_set = val_pool)

Learning rate set to 0.071005
0:	learn: 0.6102933	test: 0.6097438	best: 0.6097438 (0)	total: 30.9ms	remaining: 30.9s
1:	learn: 0.5396455	test: 0.5376447	best: 0.5376447 (1)	total: 44.2ms	remaining: 22s
2:	learn: 0.4846008	test: 0.4825247	best: 0.4825247 (2)	total: 52.9ms	remaining: 17.6s
3:	learn: 0.4389560	test: 0.4368027	best: 0.4368027 (3)	total: 60.5ms	remaining: 15.1s
4:	learn: 0.4001715	test: 0.3979680	best: 0.3979680 (4)	total: 82.7ms	remaining: 16.5s
5:	learn: 0.3690323	test: 0.3667915	best: 0.3667915 (5)	total: 100ms	remaining: 16.6s
6:	learn: 0.3438560	test: 0.3415709	best: 0.3415709 (6)	total: 109ms	remaining: 15.5s
7:	learn: 0.3216339	test: 0.3190762	best: 0.3190762 (7)	total: 121ms	remaining: 15s
8:	learn: 0.3014963	test: 0.2978330	best: 0.2978330 (8)	total: 139ms	remaining: 15.3s
9:	learn: 0.2851202	test: 0.2812826	best: 0.2812826 (9)	total: 169ms	remaining: 16.8s
10:	learn: 0.2696478	test: 0.2651098	best: 0.2651098 (10)	total: 191ms	remaining: 17.2s
11:	learn: 0.2611625	

94:	learn: 0.1543859	test: 0.1370516	best: 0.1370516 (94)	total: 2.89s	remaining: 27.6s
95:	learn: 0.1542782	test: 0.1369699	best: 0.1369699 (95)	total: 2.93s	remaining: 27.6s
96:	learn: 0.1541290	test: 0.1368277	best: 0.1368277 (96)	total: 2.96s	remaining: 27.6s
97:	learn: 0.1540235	test: 0.1367283	best: 0.1367283 (97)	total: 3.01s	remaining: 27.7s
98:	learn: 0.1540130	test: 0.1367357	best: 0.1367283 (97)	total: 3.04s	remaining: 27.6s
99:	learn: 0.1538844	test: 0.1366351	best: 0.1366351 (99)	total: 3.08s	remaining: 27.8s
100:	learn: 0.1538627	test: 0.1366280	best: 0.1366280 (100)	total: 3.12s	remaining: 27.7s
101:	learn: 0.1538465	test: 0.1366102	best: 0.1366102 (101)	total: 3.15s	remaining: 27.7s
102:	learn: 0.1537935	test: 0.1365734	best: 0.1365734 (102)	total: 3.19s	remaining: 27.8s
103:	learn: 0.1537262	test: 0.1365485	best: 0.1365485 (103)	total: 3.23s	remaining: 27.8s
104:	learn: 0.1536823	test: 0.1365459	best: 0.1365459 (104)	total: 3.26s	remaining: 27.8s
105:	learn: 0.1536309	

189:	learn: 0.1465355	test: 0.1339208	best: 0.1338873 (187)	total: 6.4s	remaining: 27.3s
190:	learn: 0.1464396	test: 0.1338458	best: 0.1338458 (190)	total: 6.44s	remaining: 27.3s
191:	learn: 0.1463521	test: 0.1338597	best: 0.1338458 (190)	total: 6.48s	remaining: 27.3s
192:	learn: 0.1463120	test: 0.1338452	best: 0.1338452 (192)	total: 6.51s	remaining: 27.2s
193:	learn: 0.1462493	test: 0.1338409	best: 0.1338409 (193)	total: 6.55s	remaining: 27.2s
194:	learn: 0.1461303	test: 0.1338385	best: 0.1338385 (194)	total: 6.59s	remaining: 27.2s
195:	learn: 0.1461023	test: 0.1338373	best: 0.1338373 (195)	total: 6.62s	remaining: 27.2s
196:	learn: 0.1460851	test: 0.1338505	best: 0.1338373 (195)	total: 6.67s	remaining: 27.2s
197:	learn: 0.1460341	test: 0.1338093	best: 0.1338093 (197)	total: 6.7s	remaining: 27.1s
198:	learn: 0.1459501	test: 0.1337806	best: 0.1337806 (198)	total: 6.74s	remaining: 27.1s
199:	learn: 0.1457813	test: 0.1337487	best: 0.1337487 (199)	total: 6.82s	remaining: 27.3s
200:	learn: 

284:	learn: 0.1399054	test: 0.1329117	best: 0.1328059 (266)	total: 10.5s	remaining: 26.3s
285:	learn: 0.1398379	test: 0.1328611	best: 0.1328059 (266)	total: 10.5s	remaining: 26.3s
286:	learn: 0.1397647	test: 0.1329042	best: 0.1328059 (266)	total: 10.6s	remaining: 26.3s
287:	learn: 0.1396928	test: 0.1328756	best: 0.1328059 (266)	total: 10.6s	remaining: 26.2s
288:	learn: 0.1395781	test: 0.1328663	best: 0.1328059 (266)	total: 10.6s	remaining: 26.2s
289:	learn: 0.1394911	test: 0.1327812	best: 0.1327812 (289)	total: 10.7s	remaining: 26.2s
290:	learn: 0.1393911	test: 0.1328030	best: 0.1327812 (289)	total: 10.7s	remaining: 26.1s
291:	learn: 0.1393473	test: 0.1328112	best: 0.1327812 (289)	total: 10.8s	remaining: 26.1s
292:	learn: 0.1393071	test: 0.1328014	best: 0.1327812 (289)	total: 10.8s	remaining: 26s
293:	learn: 0.1392616	test: 0.1328105	best: 0.1327812 (289)	total: 10.8s	remaining: 26s
294:	learn: 0.1391860	test: 0.1327843	best: 0.1327812 (289)	total: 10.9s	remaining: 26s
295:	learn: 0.13

381:	learn: 0.1342576	test: 0.1322542	best: 0.1322412 (377)	total: 14.3s	remaining: 23.1s
382:	learn: 0.1341584	test: 0.1322409	best: 0.1322409 (382)	total: 14.3s	remaining: 23.1s
383:	learn: 0.1341112	test: 0.1322575	best: 0.1322409 (382)	total: 14.4s	remaining: 23.1s
384:	learn: 0.1340842	test: 0.1322401	best: 0.1322401 (384)	total: 14.4s	remaining: 23s
385:	learn: 0.1340334	test: 0.1322389	best: 0.1322389 (385)	total: 14.4s	remaining: 23s
386:	learn: 0.1339102	test: 0.1322758	best: 0.1322389 (385)	total: 14.5s	remaining: 22.9s
387:	learn: 0.1338397	test: 0.1322723	best: 0.1322389 (385)	total: 14.5s	remaining: 22.9s
388:	learn: 0.1338396	test: 0.1322726	best: 0.1322389 (385)	total: 14.5s	remaining: 22.8s
389:	learn: 0.1338395	test: 0.1322725	best: 0.1322389 (385)	total: 14.6s	remaining: 22.8s
390:	learn: 0.1337910	test: 0.1322383	best: 0.1322383 (390)	total: 14.6s	remaining: 22.7s
391:	learn: 0.1337175	test: 0.1322045	best: 0.1322045 (391)	total: 14.6s	remaining: 22.7s
392:	learn: 0.

...

574:	learn: 0.1242432	test: 0.1314226	best: 0.1313277 (559)	total: 21.7s	remaining: 16s
575:	learn: 0.1241394	test: 0.1314191	best: 0.1313277 (559)	total: 21.7s	remaining: 16s
576:	learn: 0.1240665	test: 0.1314057	best: 0.1313277 (559)	total: 21.8s	remaining: 15.9s
577:	learn: 0.1239696	test: 0.1314850	best: 0.1313277 (559)	total: 21.8s	remaining: 15.9s
578:	learn: 0.1239330	test: 0.1314713	best: 0.1313277 (559)	total: 21.8s	remaining: 15.9s
579:	learn: 0.1239090	test: 0.1314401	best: 0.1313277 (559)	total: 21.9s	remaining: 15.8s
580:	learn: 0.1238875	test: 0.1314354	best: 0.1313277 (559)	total: 21.9s	remaining: 15.8s
581:	learn: 0.1238122	test: 0.1314067	best: 0.1313277 (559)	total: 21.9s	remaining: 15.8s
582:	learn: 0.1237204	test: 0.1314703	best: 0.1313277 (559)	total: 22s	remaining: 15.7s
583:	learn: 0.1236982	test: 0.1314499	best: 0.1313277 (559)	total: 22s	remaining: 15.7s
584:	learn: 0.1236642	test: 0.1314812	best: 0.1313277 (559)	total: 22.1s	remaining: 15.6s
585:	learn: 0.1236

668:	learn: 0.1193332	test: 0.1319047	best: 0.1313277 (559)	total: 27s	remaining: 13.4s
669:	learn: 0.1192909	test: 0.1319054	best: 0.1313277 (559)	total: 27s	remaining: 13.3s
670:	learn: 0.1192104	test: 0.1318784	best: 0.1313277 (559)	total: 27.1s	remaining: 13.3s
671:	learn: 0.1191897	test: 0.1318427	best: 0.1313277 (559)	total: 27.2s	remaining: 13.3s
672:	learn: 0.1190997	test: 0.1317845	best: 0.1313277 (559)	total: 27.3s	remaining: 13.2s
673:	learn: 0.1190689	test: 0.1317874	best: 0.1313277 (559)	total: 27.3s	remaining: 13.2s
674:	learn: 0.1189916	test: 0.1317754	best: 0.1313277 (559)	total: 27.3s	remaining: 13.2s
675:	learn: 0.1189417	test: 0.1317682	best: 0.1313277 (559)	total: 27.4s	remaining: 13.1s
676:	learn: 0.1188673	test: 0.1317503	best: 0.1313277 (559)	total: 27.4s	remaining: 13.1s
677:	learn: 0.1188124	test: 0.1317707	best: 0.1313277 (559)	total: 27.5s	remaining: 13.1s
678:	learn: 0.1187643	test: 0.1317789	best: 0.1313277 (559)	total: 27.5s	remaining: 13s
679:	learn: 0.11

763:	learn: 0.1159075	test: 0.1322916	best: 0.1313277 (559)	total: 31.2s	remaining: 9.65s
764:	learn: 0.1158897	test: 0.1322865	best: 0.1313277 (559)	total: 31.3s	remaining: 9.62s
765:	learn: 0.1158897	test: 0.1322867	best: 0.1313277 (559)	total: 31.3s	remaining: 9.57s
766:	learn: 0.1158897	test: 0.1322870	best: 0.1313277 (559)	total: 31.3s	remaining: 9.52s
767:	learn: 0.1158569	test: 0.1322990	best: 0.1313277 (559)	total: 31.4s	remaining: 9.48s
768:	learn: 0.1158457	test: 0.1322974	best: 0.1313277 (559)	total: 31.4s	remaining: 9.44s
769:	learn: 0.1157630	test: 0.1323173	best: 0.1313277 (559)	total: 31.5s	remaining: 9.4s
770:	learn: 0.1156559	test: 0.1322877	best: 0.1313277 (559)	total: 31.5s	remaining: 9.36s
771:	learn: 0.1156392	test: 0.1322847	best: 0.1313277 (559)	total: 31.6s	remaining: 9.32s
772:	learn: 0.1155638	test: 0.1322768	best: 0.1313277 (559)	total: 31.6s	remaining: 9.28s
773:	learn: 0.1155636	test: 0.1322768	best: 0.1313277 (559)	total: 31.6s	remaining: 9.24s
774:	learn:

858:	learn: 0.1121411	test: 0.1323368	best: 0.1313277 (559)	total: 35.6s	remaining: 5.85s
859:	learn: 0.1120658	test: 0.1323257	best: 0.1313277 (559)	total: 35.7s	remaining: 5.81s
860:	learn: 0.1120435	test: 0.1323193	best: 0.1313277 (559)	total: 35.7s	remaining: 5.76s
861:	learn: 0.1120228	test: 0.1323052	best: 0.1313277 (559)	total: 35.8s	remaining: 5.72s
862:	learn: 0.1119944	test: 0.1323153	best: 0.1313277 (559)	total: 35.8s	remaining: 5.68s
863:	learn: 0.1119738	test: 0.1323296	best: 0.1313277 (559)	total: 35.8s	remaining: 5.64s
864:	learn: 0.1118994	test: 0.1323168	best: 0.1313277 (559)	total: 35.9s	remaining: 5.6s
865:	learn: 0.1118617	test: 0.1323246	best: 0.1313277 (559)	total: 35.9s	remaining: 5.56s
866:	learn: 0.1118494	test: 0.1323084	best: 0.1313277 (559)	total: 36s	remaining: 5.52s
867:	learn: 0.1117676	test: 0.1323169	best: 0.1313277 (559)	total: 36s	remaining: 5.47s
868:	learn: 0.1117520	test: 0.1323258	best: 0.1313277 (559)	total: 36.1s	remaining: 5.44s
869:	learn: 0.1

951:	learn: 0.1086660	test: 0.1327662	best: 0.1313277 (559)	total: 39.8s	remaining: 2s
952:	learn: 0.1086419	test: 0.1327940	best: 0.1313277 (559)	total: 39.8s	remaining: 1.96s
953:	learn: 0.1086183	test: 0.1328030	best: 0.1313277 (559)	total: 39.8s	remaining: 1.92s
954:	learn: 0.1085854	test: 0.1327560	best: 0.1313277 (559)	total: 39.9s	remaining: 1.88s
955:	learn: 0.1085688	test: 0.1327697	best: 0.1313277 (559)	total: 39.9s	remaining: 1.84s
956:	learn: 0.1085677	test: 0.1327693	best: 0.1313277 (559)	total: 39.9s	remaining: 1.79s
957:	learn: 0.1084882	test: 0.1327822	best: 0.1313277 (559)	total: 40s	remaining: 1.75s
958:	learn: 0.1084454	test: 0.1328084	best: 0.1313277 (559)	total: 40s	remaining: 1.71s
959:	learn: 0.1084109	test: 0.1328044	best: 0.1313277 (559)	total: 40.1s	remaining: 1.67s
960:	learn: 0.1083585	test: 0.1328184	best: 0.1313277 (559)	total: 40.1s	remaining: 1.63s
961:	learn: 0.1083108	test: 0.1327942	best: 0.1313277 (559)	total: 40.1s	remaining: 1.58s
962:	learn: 0.108

In [13]:
model = CatBoostClassifier(
                            iterations=50,
                            learning_rate=0.5,
                            custom_loss=['AUC', 'Accuracy']
)

model.fit(
            train_pool,
            eval_set = val_pool,
            verbose = False,
            plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [14]:
model1 = CatBoostClassifier(
                            iterations=100,
                            learning_rate=0.7,
                            train_dir='learning_rate_0.7'
)

model2 = CatBoostClassifier(
                            iterations=100,
                            learning_rate=0.01,
                            train_dir='learning_rate_0.01'
)

model1.fit(train_pool, eval_set=val_pool, verbose=20)
model2.fit(train_pool, eval_set=val_pool, verbose=20)

0:	learn: 0.3264513	test: 0.3248170	best: 0.3248170 (0)	total: 19.9ms	remaining: 1.98s
20:	learn: 0.1688825	test: 0.1574182	best: 0.1573949 (16)	total: 239ms	remaining: 898ms
40:	learn: 0.1632884	test: 0.1582531	best: 0.1571533 (23)	total: 461ms	remaining: 663ms
60:	learn: 0.1584388	test: 0.1573279	best: 0.1569712 (52)	total: 681ms	remaining: 436ms
80:	learn: 0.1544303	test: 0.1583341	best: 0.1569712 (52)	total: 886ms	remaining: 208ms
99:	learn: 0.1510431	test: 0.1583618	best: 0.1569712 (52)	total: 1.07s	remaining: 0us

bestTest = 0.1569712214
bestIteration = 52

Shrink model to first 53 iterations.
0:	learn: 0.6853769	test: 0.6853610	best: 0.6853610 (0)	total: 18.4ms	remaining: 1.82s
20:	learn: 0.5575578	test: 0.5568257	best: 0.5568257 (20)	total: 162ms	remaining: 609ms
40:	learn: 0.4678112	test: 0.4663769	best: 0.4663769 (40)	total: 342ms	remaining: 493ms
60:	learn: 0.4029225	test: 0.4011544	best: 0.4011544 (60)	total: 480ms	remaining: 307ms
80:	learn: 0.3551621	test: 0.3530433	best:

In [15]:
from catboost import MetricVisualizer

MetricVisualizer(['learning_rate_0.7', 'learning_rate_0.01']).start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Green line is the overfitting one. It's error will increase after 50\
Blue line is underfit now, but if we increase the number of iterartion then it will properly fit

If there is overfitting then we can take the iterations till the point it gives the best or i can say min. error

In [16]:
model = CatBoostClassifier(
                            iterations=500
)

model.fit(
            train_pool,
            eval_set=val_pool,
            verbose=False,
            plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [17]:
print('Tree count = {}'.format(model.tree_count_))

Tree count = 293


# cross validation

here by default cross-validation is stratified i.e when it splittes or flods the data it make sure that each type is present in same amount in all the fold.\
In our case the datset is unbalance so we have to stratified it

In [18]:
from catboost import cv

params = {
    'loss_function': 'Logloss',
    'iterations': 200,
    'custom_loss': 'AUC',
    'learning_rate': 0.1,
    
}

cv_data = cv(
    params=params,
    pool = train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot = True,
    verbose=False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.1422216734
bestIteration = 176

Training on fold [1/5]

bestTest = 0.142385316
bestIteration = 194

Training on fold [2/5]

bestTest = 0.1532357117
bestIteration = 140

Training on fold [3/5]

bestTest = 0.1407780239
bestIteration = 184

Training on fold [4/5]

bestTest = 0.1464974816
bestIteration = 169



In [19]:
cv_data.head(10)

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-AUC-mean,test-AUC-std
0,0,0.581247,0.000047,0.581239,0.000035,0.519327,0.016610
1,1,0.493035,0.000296,0.493677,0.000183,0.574491,0.009537
2,2,0.426118,0.000150,0.427191,0.000250,0.607724,0.014645
3,3,0.377815,0.000101,0.378696,0.000211,0.607724,0.014645
4,4,0.339371,0.001135,0.340109,0.001140,0.630290,0.032263
5,5,0.308738,0.001337,0.309452,0.001114,0.674826,0.018185
6,6,0.287014,0.001882,0.287711,0.001746,0.685904,0.021819
7,7,0.267314,0.001140,0.268241,0.001155,0.726602,0.025837
8,8,0.252123,0.002173,0.253272,0.001826,0.739586,0.020460
9,9,0.237233,0.003636,0.239515,0.002467,0.772115,0.024274


In [20]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

Best validation Logloss score, not stratified: 0.1452±0.0051


In [21]:
params = {
    'loss_function': 'Logloss',
    'iterations': 200,
    'custom_loss': 'AUC',
    'learning_rate': 0.1,
    
}

cv_data = cv(
    params=params,
    pool = train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot = True,
    verbose=False,
    stratified = False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.1352226085
bestIteration = 193

Training on fold [1/5]

bestTest = 0.1516431879
bestIteration = 190

Training on fold [2/5]

bestTest = 0.1535427964
bestIteration = 129

Training on fold [3/5]

bestTest = 0.1333757494
bestIteration = 198

Training on fold [4/5]

bestTest = 0.1505907343
bestIteration = 197



# GridSearchCV

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "learning_rate": [0.01, 0.1, 0.5]
}

clf = CatBoostClassifier(
    iterations=20,
    cat_features=cat_features,
    verbose=20
)

grid_search = GridSearchCV(clf, param_grid=param_grid, cv = 5)
results = grid_search.fit(x_train, y_train)
results.best_estimator_.get_params()

0:	learn: 0.6853697	total: 16.6ms	remaining: 315ms
19:	learn: 0.5629448	total: 171ms	remaining: 0us
0:	learn: 0.6853705	total: 3.42ms	remaining: 65ms
19:	learn: 0.5629242	total: 128ms	remaining: 0us
0:	learn: 0.6853756	total: 8.42ms	remaining: 160ms
19:	learn: 0.5630543	total: 160ms	remaining: 0us
0:	learn: 0.6853683	total: 6.22ms	remaining: 118ms
19:	learn: 0.5629443	total: 73.8ms	remaining: 0us
0:	learn: 0.6853686	total: 3.77ms	remaining: 71.6ms
19:	learn: 0.5629147	total: 116ms	remaining: 0us
0:	learn: 0.6188811	total: 7.38ms	remaining: 140ms
19:	learn: 0.2303799	total: 153ms	remaining: 0us
0:	learn: 0.6188892	total: 13.6ms	remaining: 259ms
19:	learn: 0.2311997	total: 174ms	remaining: 0us
0:	learn: 0.6189377	total: 6.31ms	remaining: 120ms
19:	learn: 0.2303678	total: 114ms	remaining: 0us
0:	learn: 0.6188678	total: 8.12ms	remaining: 154ms
19:	learn: 0.2311620	total: 135ms	remaining: 0us
0:	learn: 0.6188709	total: 13.2ms	remaining: 251ms
19:	learn: 0.2322971	total: 210ms	remaining: 0us

{'iterations': 20,
 'learning_rate': 0.5,
 'verbose': 20,
 'cat_features': [0, 1, 2, 3, 4, 5, 6, 7, 8]}

# Overfitting detector

In [24]:
model_with_early_stop = CatBoostClassifier(
    iterations=200,
    learning_rate=0.5,
    early_stopping_rounds=20           # if for 20 values the values of min. error does not decrease then stop
)

model_with_early_stop.fit(
    train_pool, 
    eval_set=val_pool,
    verbose=False,
    plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Overfitting detector with eval metric

In [25]:
model_with_early_stop = CatBoostClassifier(
    iterations=200,
    eval_metric='AUC',
    learning_rate = 0.5,
    early_stopping_rounds=20
)

model_with_early_stop.fit(
    train_pool,
    eval_set=val_pool,
    verbose=False,
    plot = True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [26]:
print(model_with_early_stop.tree_count_)

47


# Predictions

In [27]:
model = CatBoostClassifier(iterations=200, learning_rate=0.03)
model.fit(train_pool, verbose=50)

0:	learn: 0.6569432	total: 23.2ms	remaining: 4.62s
50:	learn: 0.1951547	total: 670ms	remaining: 1.96s
100:	learn: 0.1664996	total: 1.58s	remaining: 1.54s
150:	learn: 0.1595258	total: 2.95s	remaining: 957ms
199:	learn: 0.1558475	total: 7.53s	remaining: 0us


In [29]:
print(model.predict(x_val))    
# this gives prediction in 0 & 1. marrgin set here will be 0.5
# not recommended to use this in classification since 0.5 will not be the best always

[1 1 1 ... 1 1 1]


In [30]:
# insted print probalities
print(model.predict_proba(x_val))

[[0.0313 0.9687]
 [0.0221 0.9779]
 [0.0109 0.9891]
 ...
 [0.0426 0.9574]
 [0.0275 0.9725]
 [0.0192 0.9808]]


In [32]:
raw_pred = model.predict(
    x_val,
    prediction_type='RawFormulaVal'  # type of prediction, here we calculating the sum of teh o/p from all trees
)

print(raw_pred)

[3.4309 3.788  4.5062 ... 3.1133 3.5647 3.9341]


In [33]:
from numpy import exp

sigmoid = lambda x: 1 / (1 + exp(-x))

probalities = sigmoid(raw_pred)
print(probalities)

[0.9687 0.9779 0.9891 ... 0.9574 0.9725 0.9808]


You can see it is matching with the previous one

#### in some cases 0.5 margine is not good